In [9]:
# fetch_20newsgroups() : 뉴스그룹의 분류 수행 예제 데이터
# 희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘 : 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등
# 텍스트 정규화 -> 피처 벡터화 -> 학습/예측/평가

In [10]:
# 텍스트 정규화

In [11]:
# fetch_20newsgroups()으로 뉴스그룹 분류 예제 데이터 불러오기. Bunch 객체 반환
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156) 

In [12]:
# news_data의 key값 확인
print(news_data.keys())
# filenames : fetch_20newsgroups() API가 인터넷에서 내려받아 로컬 컴퓨터에 저장하는 디렉터리와 파일명

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [13]:
# Target 클래스가 어떻게 구성돼 있는지 확인
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
Name: count, dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [14]:
# 개별 데이터 텍스트가 어떻게 구성돼 있는지 데이터 한 개만 추출해 확인
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [15]:
# 내용을 제외한 정보는 제거
# remove 파라미터를 이용해 뉴스그룹 기사의 헤더(header)와 푸터(footer) 등 제거 가능
# fetch_20newsgroups()의 subset 파라미터로 학습/테스트 데이터 분리가 가능

In [16]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)

X_train = train_news.data
y_train = train_news.target

# subset='test'으로 테스트 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), random_state=156)

X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


In [17]:
# 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

In [18]:
# CountVectorizer를 이용해 학습 데이터의 텍스트를 피처 벡터화 수행
# 유의할 점 : 텍스트 데이터에서 CountVectorizer를 적용 시 반드시 학습 데이터를 이용해 
# fit()이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환.
# ㄴ 그래야 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 CountVectorizer로 변환할 피처 개수가 같아짐!
# cnt_vect.transform()을 이용해 테스트 데이터 피처 벡터화 수행
# 유의할 점 : 테스트 데이터의 피처 벡터화 시 fit_transform() 사용하면 안됨. 
# ㄴ 테스트 데이터 기반으로 다시 CountVectorizer가 fit()을 수행하고 transform()하여 학습 시 사용된 피처 개수와 예측 시 사용할 피처 개수가 달라짐

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape :', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape : (11314, 101631)


In [20]:
# 피처 벡터화된 데이터에 로지스틱 회귀를 적용해 뉴스그룹에 대한 분류 예측

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# LogisticRegression을 이용하여 학습/예측/평가 수행
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.617


In [22]:
# TF-IDF 기반으로 벡터화를 변경해 예측 모델 수행

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.678


In [24]:
# TF-IDF가 일반적으로 카운터 벡터화보다 더 좋은 예측 결과를 도출
# 텍스트 분석에서 머신러닝 모델의 성능 향상 방법 : 최적읜 ML 알고리즘 선택, 최상의 피처 전처리 수행
# 텍스트 정규화, 피처 벡터화를 어떻게 효과적으로 적용했는지가 텍스트 기반의 머신러닝 성능에 큰 영향을 미칠 수 있음

In [25]:
# TfidfVectorizer 클래스의 스톱 워드를 'english'로 변경하고, ngram_range를 (1, 2)로, max_df=300으로 변경 후 다시 예측 성능 측정

In [26]:
# stop words 필터링을 추가하고 ngram을 기본 (1, 1)에서 (1, 2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.690


In [27]:
# GridSearchCV를 이용해 로지스틱 회귀의 하이퍼 파라미터 최적화 수행
# 최적의 C 값을 찾은 뒤 이 C 값으로 학습된 모델에서 테스트 데이터로 예측해 성능 평가

In [28]:
from sklearn.model_selection import GridSearchCV

# 최적의 C 값 도출 튜닝 수행. CV는 3 폴드 세트로 설정
params = {'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter :', grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter : {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.704


In [29]:
# 로지스틱 회귀의 C가 10일 때 GridSearchCV의 교차 검증 테스트 세트에서 가장 좋은 예측 성능을 나타냄
# 이를 테스트 데이터 세트에 적용해 약 0.704로 이전보다 성능 수치가 약간 향상

In [30]:
# 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합

In [31]:
# 사이킷런 Pipeline 클래스를 이용해 피처 벡터화, ML 알고리즘 학습/예측을 위한 코드 작성을 한 번에 할 수 있음
# Pipeline : 데이터의 가공, 변환 등의 전처리와 알고리즘 적용을 한꺼번에 스트림 기반으로 처리하는 것
# 데이터 전처리, 머신러닝 학습 과정을 통일된 API 기반에서 처리 가능 => 더 직관적인 ML 모델 코드 생성 가능
# 대용량 데이터의 피처 벡터화 결과를 별도 저장 없이 스트림 기반으로 바로 머신러닝 알고리즘의 데이터로 입력 가능 => 수행 시간 절약
# 사이킷런 파이프라인은 일반적으로 텍스트 기반의 피처 벡터화 뿐만 아니라 모든 데이터 전처리 작업과 Estimator 결합 가능

In [32]:
# 텍스트 분류 예제 코드를 Pipeline을 이용해 선언

In [33]:
# 파이프라인 객체 선언 방식
# TfidfVectorizer 객체를 tfidf_vect라는 객체 변수명으로, LogisticRegression 객체를 lr_clf라는 객체 변수명으로 생성한 뒤 
# 이 두 개의 객체를 파이프라인으로 연결하는 Pipeline 객체 pipeline을 생성
# pipeline = Pipeline([('tfidf_vect', TfidVectorizer(stop_words='english')), ('lr_clf', LogisticRegression(random_state=156))])

In [37]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=300)),
    ('lr_clf', LogisticRegression(solver='liblinear', C=10))
])

# 별도의 TfidfVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요 없음
# pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화의 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Pipeline을 통한 Logistic Regression의 예측 정확도는 0.704


In [41]:
# GridSearchCV 클래스의 생성 파라미터로 Pipeline을 이용한 하이퍼 파라미터 튜닝도 가능
# ㄴ> 피처 벡터화를 위한 파라미터, ML 알고리즘의 하이퍼 파라미터를 모두 한 번에 GridSearchCV로 최적화 가능

In [43]:
# GridSearchCV에 Pipeline을 입력하면서 TfidfVectorizer의 파라미터와 LogisticRegression의 하이퍼 파라미터를 함께 최적화
# GridSearchCV에 Pipeline을 입력하면 param_grid 입력 값 설정이 딕셔너리 형태.
# ㄴ Value를 리스트 형태로 입력, Key를 'tfidf_vect__ngram_range'와 같이 하이퍼 파라미터명이 객체 변수명과 결합돼 제공
# ㄴ> Pipeline 구성 파라미터와 Estimator 하이퍼 파라미터를 구별하고자 개별 객체 명, 파라미터명/하이퍼 파라미터명을 결합해 Key 값으로 할당하는 것
# Pipeline + GridSearchCV 적용 시 유의할 점 : 모두의 파라미터를 최적화하려면 너무 많은 튜닝 시간이 소모

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_) 2개를 연달아 붙여 GridSearchCV에 사용됨
# 파라미터/하이퍼 파라미터 이름과 값을 설정
params = { 'tfidf_vect__ngram_range' : [(1, 1), (1, 2), (1, 3)],
           'tfidf_vect__max_df' : [100, 300, 700],
           'lr_clf__C' : [1, 5, 10]
         }

# GridSerachCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

In [ ]:
# max_df가 700, ngram_range가 (1, 2)로 피처 벡터화된 데이터 세트에 LogisticRegression의 C에 10을 적용할 때 가장 좋은 검증 세트 성능 수치